In [3]:
import time
import pandas as pd
import os
import numpy as np
import snapatac2_scooby as sp
import scanpy as sc
import anndata as ad
import glob

import scipy.sparse
import tqdm

Important:
* SnapATAC automatically deletes reads that have the same start end and UMI (duplicates)
* If you do not provide UMI identifier (umi_regex or umi_tag) it will delete all reads that have the same start/end (not ideal)
* Our bam file reader ignores insertions or deletions in the read which can lead to small artifacts in the coverage

## Read in RNA bam files

In [4]:
data_path = '/data/ceph/hdd/project/node_08/QNA/scborzoi/submission_data'

In [ ]:
adata = sc.read(
    os.path.join(data_path, 'onek1k_training_data', 'OneK1K_only_immune.h5ad'), backed='r')

### Make Fragment file

See `process_snapatac.py`

## Combine both samples

In [4]:
import anndata 
import os
import glob

In [5]:
sample_files = glob.glob(os.path.join(data_path , 'snapatac/anndata/*pool*plus*'))

In [6]:
samples = [os.path.basename(sample_file).split('_')[1] for sample_file in sample_files]

In [8]:
for strand in [ 'minus']:
    print(strand)
    adatas = np.sort([
        os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_{strand}.h5ad') 
     for sample in samples
    ])

    anndata.experimental.concat_on_disk(
        in_files=adatas, 
        out_file=os.path.join(data_path, 'snapatac','anndata', f'snapatac_merged_{strand}.h5ad'),
    )

    # uns merge is not working, add that

    adata_cov = sc.read_h5ad(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{strand}.h5ad'))

    test = sc.read(adatas[0])

    adata_cov.uns = test.uns.copy()

    # join obs
    adata_cov.obs = adata_cov.obs.join(adata.obs)
    
    #reorder
    adata_cov = adata_cov[adata.obs_names]
    adata_cov.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{strand}.h5ad'))

minus


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
adata.obs.tissue_label.astype(str).unique()

array(['CD4+ TCM cell', 'NK cell', 'CD4+ T cell', 'CD8+ TEM cell',
       'B cell', 'CD8+ TCM cell', 'CD4+ TEM cell', 'memory B cell', 'nan',
       'CD8+ T cell', 'CD56+ NK cell', 'CD16+ monocyte', 'MAIT cell',
       'Treg memory', 'monocyte', 'gdT cell', 'CD4+ CTL cell', 'dnT cell',
       'hematopoietic precursor cell', 'platelet', 'plasmablast',
       'dendritic cell', 'plasmacytoid dendritic cell'], dtype=object)

## Write pseudobulks

In [7]:
for strand in ['plus', 'minus']:
    adata_cov = sc.read(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{strand}.h5ad'))
    adata_cov.obs = adata.obs
    adata_cov.obs.tissue_label = adata_cov.obs.tissue_label.astype(str)
    sp.ex.export_coverage(
        adata_cov, 
        groupby='tissue_label', 
        bin_size=1, 
        out_dir=os.path.join(data_path, 'snapatac',"pseudobulks"), 
        normalization=None,
        n_jobs=-1,
        max_frag_length=None,
        suffix='.bw',
        prefix=f"{strand}."
    )

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2024-12-18 13:55:11 - INFO - Exporting fragments...
2024-12-18 15:39:36 - INFO - Creating coverage files...
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transfor

In [12]:
8

8

In [11]:
3

3